## Inicializar la Dimensión Cliente
Importaciones

In [27]:
import pandas as pd
from sqlalchemy import create_engine
import yaml
from datetime import  timedelta

# Añadir conexión a la base de datos

In [28]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Consulta para extraer la información de servicios con sus novedades

In [38]:
query = """
SELECT 
    ns.*,
    ms.id as servicio_mensajeria_id,
    ms.descripcion,
    ms.nombre_solicitante,
    ms.fecha_solicitud,
    ms.hora_solicitud,
    ms.fecha_deseada,
    ms.hora_deseada,
    ms.nombre_recibe,
    ms.telefono_recibe,
    ms.descripcion_pago,
    ms.ida_y_regreso,
    ms.activo,
    ms.novedades,
    ms.cliente_id,
    ms.mensajero_id as mensajero_id_ms,  -- Renombrar esta columna
    ms.tipo_pago_id,
    ms.tipo_servicio_id,
    ms.tipo_vehiculo_id,
    ms.usuario_id,
    ms.prioridad,
    ms.ciudad_destino_id,
    ms.ciudad_origen_id,
    ms.hora_visto_por_mensajero,
    ms.visto_por_mensajero,
    ms.descripcion_multiples_origenes,
    ms.mensajero2_id,
    ms.mensajero3_id,
    ms.multiples_origenes,
    ms.asignar_mensajero,
    ms.es_prueba,
    ms.descripcion_cancelado,
    cu.sede_id
FROM
    public.mensajeria_novedadesservicio ns
LEFT JOIN
    public.mensajeria_servicio ms 
    ON ns.servicio_id = ms.id 
LEFT JOIN
    public.clientes_usuarioaquitoy cu 
    ON ms.usuario_id = cu.id
ORDER BY ns.id ASC;
"""

df_datamart = pd.read_sql(query, ryf_conn)
df_datamart.head()


,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,servicio_mensajeria_id,descripcion,nombre_solicitante,...,hora_visto_por_mensajero,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,sede_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7


# LLamar las dimensiones para poder hacer la transformación

In [39]:
df_dim_cliente = pd.read_sql('SELECT * FROM dim_cliente', etl_conn)
df_dim_mensajero = pd.read_sql('SELECT * FROM dim_mensajero', etl_conn)
df_dim_sede = pd.read_sql('SELECT * FROM dim_sede', etl_conn)
df_dim_hora = pd.read_sql('SELECT * FROM dim_hora', etl_conn)
df_dim_fecha = pd.read_sql('SELECT * FROM dim_fecha', etl_conn)


# cambiar cliente info por dim cliente

In [40]:
# Merge para obtener key_dim_cliente basado en cliente_id
df_datamart = df_datamart.merge(df_dim_cliente[['cliente_id', 'key_dim_cliente']],
                                on='cliente_id',
                                how='left')
# Eliminar la columna cliente_id original
df_datamart.drop(columns=['cliente_id'], inplace=True)

# Renombrar la columna a cliente_id si es necesario
df_datamart.rename(columns={'key_dim_cliente': 'cliente_id'}, inplace=True)

df_datamart.head()

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,servicio_mensajeria_id,descripcion,nombre_solicitante,...,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,sede_id,cliente_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7


# cambiar sede info por dim sede

In [41]:
df_dim_sede.head()

,key_dim_sede,sede_id,sede_nombre,sede_direccion,sede_telefono,contacto_sede,ciudad_nombre,cliente_nombre
0,0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
1,1,11,REMEDIOZ/ 123,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
2,2,13,DIME / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
3,3,14,DESPACHOS / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
4,4,23,POPAYAN BODEGA 28 / A,Los angeles distrito Latino,310-70000,JUAN PEREZ,POPAYAN,CARROS DEL PACIFICO (CHINA)


In [42]:
# Merge para obtener key_dim_sede basado en sede_id
df_datamart = df_datamart.merge(df_dim_sede[['sede_id', 'key_dim_sede']],
                                on='sede_id',
                                how='left')
# Eliminar la columna sede_id original
df_datamart.drop(columns=['sede_id'], inplace=True)

# Renombrar la columna a sede_id si es necesario
#df_datamart.rename(columns={'key_dim_sede': 'sede_id'}, inplace=True)

df_datamart.head()

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,servicio_mensajeria_id,descripcion,nombre_solicitante,...,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,cliente_id,key_dim_sede
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17


In [43]:
# Merge para obtener key_dim_cliente basado en cliente_id
df_datamart = df_datamart.merge(df_dim_cliente[['cliente_id', 'key_dim_cliente']],
                                on='cliente_id',
                                how='left')
# Convertir key_dim_cliente a entero inmediatamente después del merge
df_datamart['key_dim_cliente'] = df_datamart['key_dim_cliente'].astype(int)
# Eliminar la columna cliente_id original
df_datamart.drop(columns=['cliente_id'], inplace=True)



In [44]:
df_datamart.head()

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,servicio_mensajeria_id,descripcion,nombre_solicitante,...,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,key_dim_sede,key_dim_cliente
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8


# cambiar mensajero info por dim mensajero

In [45]:
# Merge para obtener key_dim_mensajero basado en mensajero_id
df_datamart = df_datamart.merge(df_dim_mensajero[['mensajero_id', 'key_dim_mensajero']],
                                on='mensajero_id',
                                how='left')

# Convertir key_dim_mensajero a entero inmediatamente después del merge
df_datamart['key_dim_mensajero'] = df_datamart['key_dim_mensajero'].astype(int)

# Eliminar la columna mensajero_id original
df_datamart.drop(columns=['mensajero_id'], inplace=True)



In [46]:
df_datamart.head()


,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,servicio_mensajeria_id,descripcion,nombre_solicitante,fecha_solicitud,...,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,key_dim_sede,key_dim_cliente,key_dim_mensajero
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37


# cambiar fecha info por dim fecha

In [53]:
# Convertir fecha_solicitud a formato datetime64[ns]
df_datamart['fecha_solicitud'] = pd.to_datetime(df_datamart['fecha_solicitud']).dt.date

# Merge para fecha_solicitud
df_datamart = df_datamart.merge(df_dim_fecha[['fecha_completa', 'key_dim_fecha']],
                                left_on='fecha_solicitud',
                                right_on='fecha_completa',
                                how='left')
df_datamart.rename(columns={'key_dim_fecha': 'key_dim_fecha_solicitud'}, inplace=True)
df_datamart.drop(columns=['fecha_completa'], inplace=True)


MemoryError: Unable to allocate 80.5 GiB for an array with shape (10799308800,) and data type int64

In [48]:
df_datamart.head()

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,servicio_mensajeria_id,descripcion,nombre_solicitante,fecha_solicitud,...,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,key_dim_sede,key_dim_cliente,key_dim_mensajero
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
